## **<center><font color=red>Faculty Development Program</font></center>** 

**<center>Electronics and ICT Academy**
**<br><center>Indian Institute of Technology, Roorkee**
    
**<center><font color=green>"Practical Quantum Computing using Qiskit and IBMQ"</font></center>**

## **Project Work**

**B92 Quantum Key Distribution Protocol**

## The B92 Quantum Key Distribution Protocol

A Gentle Introduction_ by Eleanor Rieffel and Wolfgang Polak. 
<br>This protocol is different from BB84 and was proposed by Charles Bennett in 1992. We will consider the version of the protocol without eavesdropping. As before, there are two parties, Alice and Bob. 
They communicate via a unidirectional quantum channel from Alice to Bob, and a authenticated bidirectional classical communication channel. The setup is shown in the figure below:

![QKD Setup](https://raw.githubusercontent.com/deadbeatfour/quantum-computing-course/master/img/qkd.png)

In this protocol, Alice and Bob generate one random binary string each. Alice encodes qubits according to values of her random binary string. For each bit in her binary string, she encodes $0$ as the $|0\rangle$ state and encodes $1$ as the $|+\rangle$ state and then sends all the qubits to Bob. Bob measures the qubits by choosing bases according to his random binary string. If the $i^{th}$ bit of his string is $0$, he measures the $i^{th}$ qubit in the Hadamard basis. If the $i^{th}$ bit is $1$, he measures the $i^{th}$ qubit in the computational basis. 

Finally Bob announces the results of his measurements over the classical channel. Alice and Bob keep only those bits from their binary strings corresponding to the qubits for which Bob measured an outcome of 1 to obtain their keys. The steps are mentioned in detail in the sections below:

## Choosing bases and encoding states

Alice generates one binary string and encodes her qubits using the following scheme:

$0  \rightarrow |0\rangle$

$1  \rightarrow |+\rangle$ 

Bob also generates a binary string and uses the following convention to choose a basis for measurement

$0 \rightarrow$ Hadamard basis

$1 \rightarrow$ Computational basis



**Importing Standard python and Qiskit Libraries needed for the work**

In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

from qiskit import QuantumCircuit, execute
from qiskit.providers.aer import QasmSimulator
from qiskit.visualization import *
import qiskit
from packaging.version import parse as parse_version
assert parse_version(np.__version__) >= parse_version('1.19'),  "Please install the correct version of numpy using the command 'pip install --upgrade numpy==1.19'"
assert parse_version(qiskit.__qiskit_version__['qiskit-terra']) >= parse_version('0.15'), "Please make sure you have the correct version of Qiskit installed or run this on IBM Quantum Experience" 
assert parse_version(qiskit.__qiskit_version__['qiskit-aer']) >= parse_version('0.6'), "Please make sure you have the correct version of Qiskit installed or run this on IBM Quantum Experience" 
assert parse_version(qiskit.__qiskit_version__['qiskit']) >= parse_version('0.20'),"Please make sure you have the correct version of Qiskit installed or run this on IBM Quantum Experience" 
from cryptography.fernet import Fernet
import base64
basis_gates = ['id', 'x', 'y', 'z', 's', 't', 'sdg', 'tdg', 'h', 'p', 'sx' ,'r', 'rx', 'ry', 'rz', 'u', 'u1', 'u2', 'u3', 'cx', 'barrier', 'measure']

**THE SECRET MESSAGE**

In [2]:
secret_message = b'gAAAAABfevgMDRKfpM75bCBMUfAvaUW_Fjs2PxFYkYOSCldJTUnl8oLKVZRaiPitXqwQwbMTx4YwSCf_n0HQ-RIBvLa58AN4Pi7Fp9hFxGtjwzIpWUXIUr-BGE_9SLvjUGgsQCyrhK9ZJ5Yy9R5F6w4Me0Csr19UU3IqQQIP3ffhInE5o68_CI_URCjHXpBUnztJoDmlBnZz3Ka5NykfUN22iulaFvXOyw=='
print(f"The secret message is {secret_message.decode()}")

The secret message is gAAAAABfevgMDRKfpM75bCBMUfAvaUW_Fjs2PxFYkYOSCldJTUnl8oLKVZRaiPitXqwQwbMTx4YwSCf_n0HQ-RIBvLa58AN4Pi7Fp9hFxGtjwzIpWUXIUr-BGE_9SLvjUGgsQCyrhK9ZJ5Yy9R5F6w4Me0Csr19UU3IqQQIP3ffhInE5o68_CI_URCjHXpBUnztJoDmlBnZz3Ka5NykfUN22iulaFvXOyw==


In the cell below, we generate two random binary strings for Alice and Bob respectively. These will be used by Alice to encode her state, and by Bob to decide his measurement bases. Since this is a standardised assignment, we have seeded the random number generator to produce the same output every time you run the cell below. We have used this setup and a symmetric key cipher to encrypt a secret message (the ciphertext was printed after the cell above). Your goal in this exercise is to complete the B92 Protocol correctly and discover the secret message. 

In [3]:
num_qubits = 64

rng = np.random.default_rng(seed=10)

alice_state = rng.integers(0, 2, size=num_qubits)
bob_basis = rng.integers(0, 2, size=num_qubits)

print(f"Alice's State:\t {np.array2string(alice_state, separator='')}")
print(f"Bob's Bases:\t {np.array2string(bob_basis, separator='')}")

Alice's State:	 [1100111011000101001101001111110110110001100110010110101100110000]
Bob's Bases:	 [1101001000000011110100000000011011001001111111110011010000111001]


## Creating the circuit

Based on the following result:

$H|0\rangle = |+\rangle$

Our algorithm to construct the circuit is as follows:

1. Whenever Alice wants to encode `1` in a qubit, she applies an $H$ gate to the qubit. To encode `0`, no action is needed.

2. She then _sends_ the qubits to Bob (symbolically represented in this circuit using wires)

3. Bob measures the qubits according to his binary string. To measure a qubit in the Hadamard basis, he applies an $H$ gate to the corresponding qubit and then performs a standard basis measurement. 


Given below is the structure for a function `make_b92_circ(enc_state, meas_basis)` which returns a `QuantumCircuit()` to simulate the B92 QKD protocol. The task is to implement steps 1 through 3 above and populate the function below. For step 3, we  need to apply the gate to change the basis.

The method is the same as was used for BB84. 

In [7]:
def make_b92_circ(enc_state, meas_basis):
    '''
    A function that makes a B92 QKD protocol simulation circuit
    
    enc_state: array of 0s and 1s denoting the state to be encoded using the following scheme:
                0 -> |0>
                1 -> |+>
    
    meas_basis: array of 0s and 1s denoting the basis to be used for measurement
                0 -> Hadamard Basis
                1 -> Computational Basis
    
    Note that both enc_state and meas_basis are arrays of integers, so if you are using them in 
    if statements, compare them to integer values like 0 and 1 (without quotes).
    
    Since this is a function, you only have access to the variables enc_state and meas_basis.
    You may define other local variables. One such variable, num_qubits has been defined for you. 
    This is the number of qubits in the B92 simulation QuantumCircuit()
    '''
    num_qubits = len(enc_state)

    b92 = QuantumCircuit(num_qubits)

    # Sender prepares qubits
    # To encode the state in qubits
    for index in range(len(enc_state)):
        if enc_state[index] == 1:
            b92.x(index)
            b92.h(index)  
    b92.barrier()  
    # Receiver measures the received qubits
    # To change basis for measurements. 
    for index in range(len(meas_basis)):
        if meas_basis[index] == 0:
            b92.h(index)
            b92.x(index)

    
    b92.measure_all()
    return b92

## Simulating B92
Once we populated the function above, run the cell below to check if the function works correctly. We have added some basic checks and we are checking your measurement results against the solution. 

<div class="alert alert-block alert-danger"><b>Note:</b> This code is designed by us to check whether your function works correctly. </div>

The result of Bob's measurements are also printed below.

In [9]:
try:
    b92_circ = make_b92_circ(alice_state, bob_basis)
    assert list(b92_circ.count_ops()) != [], "Circuit cannot be empty"
    assert set(b92_circ.count_ops().keys()).difference(basis_gates) == set(), f"Only the following basic gates are allowed: {basis_gates}"
    assert all([type(gate[0]) == qiskit.circuit.measure.Measure for gate in b92_circ.data[-b92_circ.num_qubits:len(b92_circ.data)]]), "Measurement must be the last operation in a circuit."
    assert b92_circ.count_ops()['measure'] == b92_circ.num_qubits, "Please do not add or remove measurements."
    temp_key = execute(
        b92_circ.reverse_bits(),
        backend=QasmSimulator(),
        shots=1, 
        seed_simulator=10
    ).result().get_counts().most_frequent()
    assert temp_key == bin(16228741048440553634)[2:], "Your circuit did not perform as expected. Please check the gates again."
    print(f"Bob's results:\t{temp_key}\nYour answer is correct.")
except AssertionError as e:
    print(f'Your code has an error:  {e.args[0]}')
except Exception as e:
    print(f'This error occured: {e.args[0]}')

Bob's results:	1110000100111000000100100000000000000000000110011000000010100010
Your answer is correct.


## Creating the Key (Sifting)

Now we need to generate the key via sifting. The sifting process for B92 is different from that of BB84. 
After Bob has measured the qubits, Bob will announce his measured result (the binary string printed after the previous cell). Then, Alice and Bob keep the bits in their randomly generated binary strings at the positions where Bob measured an outcome `1` in his result. Then both he and Alice discard all other bits from their respective strings. 

Given below is the structure for a function `b92_sifting(enc_state, meas_basis, meas_result)`. This function will perform key sifting based on Bob's measurement results. 
Inside the function there are two variables `sender_key` and `receiver_key`. The names are self explanatory. The sifting process is given below:

Loop through each character in the `meas_result` argument. For the $i^{th}$ character:

1. If the measured outcome is `'1'`, 
    - Append the $i^{th}$ bit from the `enc_state` argument to the `sender_key`
    - Append the $i^{th}$ bit from the `meas_basis` argument to the `receiver_key`
    
2. If the measured outcome is `'0'`, 
    - Do nothing. 

In [11]:
def b92_sifting(enc_state, meas_basis, meas_result):
    '''
    The function that implements key sifting for the B92 QKD protocol.
    
    enc_state:  array of 0s and 1s denoting the state to be encoded.
                (Array of integers)
    
    meas_basis: array of 0s and 1s denoting the basis to be used for measurement.
                (Array of integers)
    
    meas_result: A string of characters representing the results of measurement after the 
                B92 QKD protocol. Note that this is a string and its elements are characters, 
                so while using any if statements, compare the elements to '0' and '1' (with quotes)
                
    Since this is a function, you only have access to the variables enc_state, meas_basis and meas_result.
    You may define other local variables. num_qubits has been defined for you. 
    This is the number of qubits in the B92 simulation QuantumCircuit.
    
    sender_key and receiver_key are initialised as two empty strings. You may append bits using the += 
    operation as shown in the BB84 notebook. Note that you can only add characters. To change from other 
    data types to characters, you may use str(). Check the BB84 notebook for examples. 
    '''
    
    num_qubits = len(enc_state)

    sender_key = ''
    receiver_key = ''
    
    # Loop over all bits in the meas_result string and add the necessary bits to both sender_key and receiver_key
    # Add your code below
    for i in range(len(meas_result)):
        if meas_result[i]=='1':
            sender_key += str(enc_state[i])
            receiver_key += str(meas_basis[i])
        else:
            pass
            
   
    return (sender_key, receiver_key)

# Obtaining the Final Key and Decrypting the Message
Once you have filled in the function above, run the following cell. We use the function you filled to obtain the final sifted key from Alice's and Bob's binary strings and Bob's measurement results. 
Those keys are printed. If all goes well, the secret message will also be decrypted for you. 

In [12]:
try: 
    alice_key, bob_key = b92_sifting(alice_state, bob_basis, temp_key)
    assert ''.join([str(x ^ y) for x, y in zip(alice_key.encode(), bob_key.encode())]) != '1'*len(alice_key), "Please check your measurement convention"
    assert alice_key == bob_key, "They keys are different for Alice and Bob."
    assert alice_key == bob_key == bin(49522)[2:], "They keys is incorrect. Please check your solutions."
    print(f"Alice's Key: \t{alice_key}\nBob's Key: \t{bob_key}\nYour answer is correct.")
    g = Fernet(base64.b64encode(bob_key.encode()*2))
    print(f"The secret message is: {g.decrypt(secret_message).decode()}")
except AssertionError as e:
    print(f'Your code has an error:  {e.args[0]}')
except Exception as e:
    print(f'This error occured: {e.args[0]}')  

Alice's Key: 	1100000101110010
Bob's Key: 	1100000101110010
Your answer is correct.
The secret message is: 
Thank you for participating in the course. We hope you had fun. 
					-With ❤️ from IIT Roorkee
